In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%  cd /content/drive/My Drive/Sentiment_analysis

In [10]:
import os
import string
import numpy as np
import pandas as pd
from numpy import array

import gensim
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, GRU
from keras.initializers import Constant



In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Biswajit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Biswajit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
df = pd.read_csv('data/movie_data.csv')
df.head()

FileNotFoundError: [Errno 2] File b'data/movie_data.csv' does not exist: b'data/movie_data.csv'

In [15]:
review_lines = list()
lines = df['review'].values.tolist()

for line in lines:
    tokens = word_tokenize(line)

    # convert into lower case
    tokens = [w.lower() for w in tokens]

    # remove punctuation from each word
    table = str.maketrans(" ", " ", string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    # remove remaining tokens that are not alhabetic
    words = [word for word in stripped if word.isalpha()]

    #filter out stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    review_lines.append(words)
    # print(tokens)
    # print(table)
    # print(stripped)
    # print(words)
    # break

In [16]:
len(review_lines)

50000

In [17]:
EMBEDDING_DIM = 100
model = gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDING_DIM, window=5, workers=4, min_count=1)



In [18]:
#vocab size
words = list(model.wv.vocab)
print("Vocabulary size: %d" % len(words))

Vocabulary size: 134095


In [19]:
model.wv.most_similar('horrible')

[('terrible', 0.9254419803619385),
 ('awful', 0.8556784987449646),
 ('horrendous', 0.8362759351730347),
 ('atrocious', 0.7899037599563599),
 ('pathetic', 0.7796226739883423),
 ('dreadful', 0.7775373458862305),
 ('horrid', 0.7713038921356201),
 ('sucks', 0.7699170112609863),
 ('lousy', 0.768580973148346),
 ('bad', 0.7498810291290283)]

In [20]:
# let's see the result of semantically reasonale word vectors (King - man + woman)
model.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])


[('princess', 0.8458237648010254),
 ('juliet', 0.8339255452156067),
 ('cortland', 0.8289687037467957),
 ('carmen', 0.8268960118293762),
 ('tearstained', 0.8166431188583374),
 ('nearbiographical', 0.8151580095291138),
 ('emissaries', 0.8135117888450623),
 ('jangha', 0.8110916614532471),
 ('lion', 0.8102676272392273),
 ('bride', 0.806472659111023)]

In [21]:
# odd word out
print(model.wv.doesnt_match("woman king queen movie".split()))


movie


In [22]:
embeddings_index = {}
f = open(os.path.join("",'data/imdb_embedding_word2vec.txt'), encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'data/imdb_embedding_word2vec.txt'

In [ ]:
# os.path.join("",'data/imdb_embedding_word2vec.txt')
embeddings_index['movie']

In [ ]:
# Vectorize the text samples into a 2d integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines)



In [ ]:
#pad sequences
word_index = tokenizer_obj.word_index
print("Found %s unique tokens." % len(word_index))


In [ ]:
# pad sequence
max_length = max([len(s) for s in total_reviews])

# Define vocabulary size
# vocab_size = len(tokenizer_obj.word_index) + 1


In [ ]:
review_pad = pad_sequences(sequences=sequences, maxlen=max_length)
sentiment = df['sentiment'].values
print("Shape of review tensor:", review_pad.shape)
print("Shape of sentiment tensor:", sentiment.shape)

In [ ]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
#         words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
print(num_words)

In [ ]:
# Define model
model = Sequential()
embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=max_length,
    trainable=False
)
model.add(embedding_layer)
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
print(model.summary())


In [ ]:
# Split the date into training set and validation set
VALIDATION_SPLIT = 0.2

indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int(VALIDATION_SPLIT*review_pad.shape[0])

X_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

In [ ]:
print("Shape of X_train_pad tensor", X_train_pad.shape)
print("Shape of y_train tensor", y_train.shape)
print("Shape of X_test_pad tensor", X_test_pad.shape)
print("Shape of y_test tensor", y_test.shape)

In [ ]:
print("Train .....")

model.fit(X_train_pad, y_train,  batch_size=128, epochs=10, validation_data=(X_test_pad, y_test), verbose=2)


In [ ]:
# Saving the model
model.save('data/SA_w2v_GRU_model.h5')

# Saving the weights of model
model.save_weights('checkpoints/sa_w2v_gru_checkpoint.ckpt')
